In [ ]:
from pyeasyga import pyeasyga #used for implementation of genetic algorithm
from sumolib import checkBinary #sumolib is a set of python modules for working with sumo networks, simulation output and other simulation artifacts.
#checkBinary(name, bindir=None): Checks for the given binary in the places, defined by the environment variables SUMO_HOME and <NAME>_BINARY.
import xml.etree.ElementTree as ET #The xml.etree.ElementTree module implements a simple and efficient API for parsing and creating XML data.
import traci #Traffic Control Interface
import sys #This module provides access to some variables used or maintained by the interpreter and to functions that interact strongly with the interpreter

In [ ]:
def simulate(program):
    traci.start([checkBinary(program), "-c", "sumo/light.sumocfg", "--tripinfo-output", "sumo/tripinfo.xml"]) #sumo gets started
    
    while traci.simulation.getMinExpectedNumber() > 0:  #Returns the number of all active vehicles and persons which are in the net plus the ones still waiting to start.
        traci.simulationStep() #used to strt simultion
        
    traci.close() #simultion is closed 
    sys.stdout.flush()  #Calling sys.stdout.flush() forces it to "flush" the buffer, meaning that it will write everything in the buffer to the terminal, even if normally it would wait before doing so.
    
    xmldoc = ET.parse('sumo/tripinfo.xml')  #parse the xml file

    tripinfos = xmldoc.findall('tripinfo')  #Element.findall() finds only elements with a tag which are direct children of the current element.

    waitingTime = 0
    for tripinfo in tripinfos:
        waitingTime += float(tripinfo.get('waitingTime'))  #The time in which the vehicle speed was below or equal 0.1 m/s (scheduled stops do not count)
        
    return waitingTime
  #return the waitingTime

In [ ]:
def display(phenotype):
    network = ET.parse('sumo/light.net.xml')  #parse the xml file
    signal = network.find('tlLogic')  #Element.find() finds the first child with a particular tag  #tllogic trffic light logic
    phases = signal.findall('phase')   #In a SUMO-TLS definition, time is on the vertical axis and each phase describes all signal states that last for a fixed duration. 
    
    for i in range(4):
        phases[i].set("duration", str(phenotype[2*i]))
        phases[i].set("state", phenotype[2*i+1])     #model implementtion

    network.write("sumo/light.net.xml")   #build network

    return simulate('sumo-gui')  #crete gui

In [ ]:
default = [42, 'GGGGgrrrrrGGGGgrrrrr', 3, 'yyyyyrrrrryyyyyrrrrr', 42, 'rrrrrGGGGgrrrrrGGGGg', 3, 'rrrrryyyyyrrrrryyyyy']
display(default)

In [ ]:
data = [0] * 188
ga = pyeasyga.GeneticAlgorithm(data)  #implementtion of genetic model
ga.generations =  10 #Each generation consist of a population of individuals and each individual represents a point in search space and possible solution. 
ga.population_size = 20  #population size is an important parameter which directly influences the ability to search an optimum solution in the search space

fitnesses = [] 

In [2]:
#Phenotype is the population in the actual real world solution space in which solutions are represented in a way they are represented in real world
def phenotype(genotype):
    phenotype = []
    m = ['G', 'g', 'y', 'r']
    
    for j in range(4):
        phenotype.append(int(''.join(map(str,genotype[:7])),2)) 

        state = genotype[7:47]
        s = ""
        for i in range(0, 20):#population size
            s += m[int(str(state[2*i])+str(state[2*i+1]),2)]#Generating the red yellow green state sequence
        phenotype.append(s)

        genotype = genotype[47:]
    
    return phenotype

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [ ]:
#A fitness function is a particular type of objective function that is used to summarise, as a single figure of merit, how close a given design solution is to achieving the set aims. Fitness functions are used in genetic programming and genetic algorithms to guide simulations towards optimal design solutions.
def fitness(individual, data):#Individual= array of (duration,state)
    
    individual = phenotype(individual)

    network = ET.parse('sumo/light.net.xml')
    signal = network.find('tlLogic') 
    
    i = 0
    for phase in signal.iter('phase'):
        duration = str(individual[2*i])#finding Duration
        if duration == "0":
             return 0
        state = individual[2*i+1]
        phase.set("duration", duration)#setting the duration of timer
        phase.set("state", state)#Setting the state(RYG) of light
        i += 1

    network.write("sumo/light.net.xml")
    
    waitingTime = simulate('sumo')
    
    fitnesses.append(waitingTime)
          
    print(waitingTime, individual)
    
    return 1/waitingTime

ga.fitness_function = fitness

In [ ]:
ga.run()

In [ ]:
best = phenotype(ga.best_individual()[1])  
#Print the best solution

In [ ]:
from statistics import mean, stdev
import matplotlib.pyplot as plt
import numpy as np

generation_fitnesses = []
elite_fitnesses = []  #Elite(Best) fitness values from 50 repetitions of the 1000-generation evolving-mechanism (EM) experiment operating with M1.  #minimum wt
mean_fitnesses = []
stdev_fitnesses = []

for i in range(ga.generations):
    generation_fitnesses.append(fitnesses[ga.population_size*i:(ga.population_size)*(i+1)])
#Finding the best fitness(minimum),mean and standard deviation from the output of generations
for i in range(ga.generations):
    elite_fitnesses.append(min(generation_fitnesses[i]))#Best Fitness
    mean_fitnesses.append(mean(generation_fitnesses[i]))
    stdev_fitnesses.append(stdev(generation_fitnesses[i]))
#Displaying fitness values    
print(elite_fitnesses)
print(mean_fitnesses)
print(stdev_fitnesses)

#Plotting
plt.plot(fitnesses, label='Fitness')
t = np.linspace(ga.population_size/2, (ga.population_size/2)*(2*ga.generations-1), ga.generations)
plt.plot(t, elite_fitnesses,'bs', label='Elite Fitness')
plt.plot(t, mean_fitnesses, 'ro', label='Mean Fitness')
plt.plot(t, stdev_fitnesses, 'g*', label='Standard Deviation')
plt.xlabel('Generation')
plt.ylabel('Waiting Time')
plt.legend()
plt.show()

In [ ]:
# Best individual, after 10 generations each with a population of 20 individuals
display(best)

In [ ]:
# Best so far, after 30 generations each with a population of 50 individuals
best_so_far = [43, 'GGGrGgrGrgGGyyGGgGyy', 11, 'GgyGGyGyrgggrGGGGGrr', 38, 'GgGGgGrgyyGGgyygGryr', 78, 'yrrGgryyrrgGyGrGyggG']
display(best_so_far)